In [ ]:
import numpy as np
import polars as pl

import lightgbm as lgb
import optuna

import time
import math
import warnings
warnings.filterwarnings('ignore')

In [ ]:
R = 5000/50
num_brackets = math.floor(math.log(R, 3)) + 1
print('Number of brackets', num_brackets)

Number of brackets 5


In [ ]:
# optuna hyper-parameter optimization with hyperband
num_boost_round = 5000

lgb_data_eval = lgb.Dataset(
    data=eval_data_x, label=eval_data_y, free_raw_data=False)
lgb_data_valid = lgb.Dataset(
    data=valid_data_x, label=valid_data_y, reference=lgb_data_eval, free_raw_data=False)
lgb_data_test = lgb.Dataset(
    data=test_data_x, label=test_data_y, reference=lgb_data_eval)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting': 'gbdt',
        'learning_rate': 0.01,
        # 'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-4, 100.0),
        # 'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-4, 100.0),
        'num_leaves': trial.suggest_int('num_leaves', 16, 512, log=True),
        # 'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
        # 'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 1.0),
        'cat_l2': trial.suggest_uniform('cat_l2', 5, 15),
        'cat_smooth': trial.suggest_uniform('cat_smooth', 5, 15),
        'verbosity': -1,
        'seed': 0
    }

    # init model and pruning mechanism
    gbm = lgb.train(
        params=params,
        num_boost_round=50,
        train_set=lgb_data_eval,
        valid_sets=[lgb_data_valid],
        callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)],
        keep_training_booster=True
    )

    for step in range(num_boost_round):
        gbm = lgb.train(
            params=params,
            num_boost_round=num_boost_round,
            train_set=lgb_data_eval,
            valid_sets=[lgb_data_valid],
            callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)],
            keep_training_booster=True,
            init_model=gbm
        )

        intermediate_pred = gbm.predict(
            data=valid_data_x, num_iteration=gbm.best_iteration)
        intermediate_rmse = mean_squared_error(
            valid_data_y, intermediate_pred, squared=False)
        
        trial.report(intermediate_rmse, step)

        if trial.should_prune():
            raise optuna.TrialPruned()

    valid_data_pred = gbm.predict(
        data=valid_data_x, num_iteration=gbm.best_iteration)
    rmse = mean_squared_error(valid_data_y, valid_data_pred, squared=False)

    return rmse


optuna.logging.set_verbosity(optuna.logging.INFO)

sampler = optuna.samplers.TPESampler(seed=0)
pruner = optuna.pruners.HyperbandPruner(
    min_resource=50, max_resource=num_boost_round, reduction_factor=3)

study_hb = optuna.create_study(
    sampler=sampler, pruner=pruner, direction='minimize')
study_hb.optimize(objective, n_trials=20, show_progress_bar=True)

print('Number of finished trials:', len(study_hb.trials))
print('Best trial:', study_hb.best_trial.params)

In [ ]:
fig = optuna.visualization.plot_optimization_history(study_hb)
fig.show()

In [ ]:
fig = optuna.visualization.plot_intermediate_values(study_hb)
fig.show()

In [ ]:
fig = optuna.visualization.plot_contour(study_hb)
fig.show()